In [2]:
ANVIL_FHIR="https://healthcare.googleapis.com/v1/projects/ncpi-fhir-cat-2022/locations/us-central1/datasets/GTEx_Open_Access/fhirStores/gtex_v8/fhir/"
auth=sprintf("Bearer %s",system2("gcloud", args=c("auth", "print-access-token"), stdout=TRUE))

In [7]:
library(tidyverse)
library(httr)
library(jsonlite)
#Create some reading functions to query the fhir server and extract resource responses.

simple_get = function(request) {
  res=GET(request,add_headers(Authorization=auth))
  con=content(res,as="parsed",type="application/json")
  con
}

default_get=function(request) {
  simple_get(paste0(ANVIL_FHIR,request))
}


get_all=function(request) {
  next_request=paste0(ANVIL_FHIR,request)
  all_content=list()
  while(next_request!="") {
    my_content=simple_get(next_request)
    if(my_content$resourceType=="Bundle") {
      all_content=append(all_content,lapply(my_content$entry,function(x){x$resource}))
      next_request=paste0(sapply(my_content$link,function(x){ifelse(x$relation=="next",x$url,"")}),collapse = "")
    }
    else {
      all_content=append(all_content,list(my_content))
      next_request=""
    }
  }
  all_content
}



Attaching package: ‘jsonlite’


The following object is masked from ‘package:purrr’:

    flatten




In [8]:
default_get("ResearchStudy")

$entry
$entry[[1]]
$entry[[1]]$fullUrl
[1] "https://healthcare.googleapis.com/v1/projects/ncpi-fhir-cat-2022/locations/us-central1/datasets/GTEx_Open_Access/fhirStores/gtex_v8/fhir/ResearchStudy/f6c7b779-657e-4c89-9a52-d7a39f3c661d"

$entry[[1]]$resource
$entry[[1]]$resource$enrollment
$entry[[1]]$resource$enrollment[[1]]
$entry[[1]]$resource$enrollment[[1]]$reference
[1] "Group/ec345c13-53ba-4dcd-b9f3-9e38826f41fd"



$entry[[1]]$resource$id
[1] "f6c7b779-657e-4c89-9a52-d7a39f3c661d"

$entry[[1]]$resource$identifier
$entry[[1]]$resource$identifier[[1]]
$entry[[1]]$resource$identifier[[1]]$system
[1] "https://anvilproject.org/gtex/fhir/researchstudy"

$entry[[1]]$resource$identifier[[1]]$use
[1] "official"

$entry[[1]]$resource$identifier[[1]]$value
[1] "GTEx"


$entry[[1]]$resource$identifier[[2]]
$entry[[1]]$resource$identifier[[2]]$system
[1] "https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id="

$entry[[1]]$resource$identifier[[2]]$value
[1] "phs000424"



$entry[[1]]$resource$keyword
$entry[[1]]$resource$keyword[[1]]
$entry[[1]]$resource$keyword[[1]]$coding
$entry[[1]]$resource$keyword[[1]]$coding[[1]]
$entry[[1]]$resource$keyword[[1]]$coding[[1]]$code
[1] "GTEx"





$entry[[1]]$resource$meta
$entry[[1]]$resource$meta$lastUpdated
[1] "2022-06-24T20:35:19.281681+00:00"

$entry[[1]]$resource$meta$profile
$entry[[1]]$resource$meta$profile[[1]]
[1] "https://ncpi-fhir.github.io/ncpi-fhir-study-summary-ig/StructureDefinition/ncpi-research-study"


$entry[[1]]$resource$meta$tag
$entry[[1]]$resource$meta$tag[[1]]
$entry[[1]]$resource$meta$tag[[1]]$code
[1] "GTEx"

$entry[[1]]$resource$meta$tag[[1]]$system
[1] "https://anvilproject.org/gtex/fhir/researchstudy"



$entry[[1]]$resource$meta$versionId
[1] "MTY1NjEwMjkxOTI4MTY4MTAwMA"


$entry[[1]]$resource$relatedArtifact
$entry[[1]]$resource$relatedArtifact[[1]]
$entry[[1]]$resource$relatedArtifact[[1]]$type
[1] "documentation"

$entry[[1]]$resource$relatedArtifact[[1]]$url
[1] "https://gtexportal.org"



$entry[[1]]$resource$resourceType
[1] "ResearchStudy"

$entry[[1]]$resource$sponsor
$entry[[1]]$resource$sponsor$reference
[1] "Organization/7f2816f7-9385-4e85-b944-2142e410ca7a"


$entry[[1]]$resource$status
[1] "completed"


$entry[[1]]$search
$entry[[1]]$search$mode
[1] "match"




$link
$link[[1]]
$link[[1]]$relation
[1] "search"

$link[[1]]$url
[1] "https://healthcare.googleapis.com/v1/projects/ncpi-fhir-cat-2022/locations/us-central1/datasets/GTEx_Open_Access/fhirStores/gtex_v8/fhir/ResearchStudy/?"


$link[[2]]
$link[[2]]$relation
[1] "first"

$link[[2]]$url
[1] "https://healthcare.googleapis.com/v1/projects/ncpi-fhir-cat-2022/locations/us-central1/datasets/GTEx_Open_Access/fhirStores/gtex_v8/fhir/ResearchStudy/?"


$link[[3]]
$link[[3]]$relation
[1] "self"

$link[[3]]$url
[1] "https://healthcare.googleapis.com/v1/projects/ncpi-fhir-cat-2022/locations/us-central1/datasets/GTEx_Open_Access/fhirStores/gtex_v8/fhir/ResearchStudy/?"



$resourceType
[1] "Bundle"

$total
[1] 1

$type
[1] "searchset"

In [11]:
all_files=get_all("DocumentReference?category=RNA-Seq&location:missing=false")

In [23]:
length(all_files)

[1] 17382

In [25]:
df = as_tibble_col(all_files,"resource")

In [26]:
dim(df)

[1] 17382     1

In [29]:
stream_out(df, file(tmp <- tempfile()))

opening file output connection.



Complete! Processed total of 17382 rows.


closing file output connection.



In [30]:
print(tmp)

[1] "/tmp/RtmpxCH1B1/file82768169d3"


In [31]:
system2("gsutil", args=c("cp", tmp, "$WORKSPACE_BUCKET/DocumentReference/gtex.ndjson"))